In [20]:
%load_ext autoreload
%autoreload 2
import glob2
import pandas as pd
import numpy as np
import text_summarization
import extract_text
import sentiment_analyzer
import keras
from tqdm import tqdm, tqdm_pandas
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import warnings
import csv
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
df_files = pd.read_csv("df_file_final.csv", lineterminator='\n')

In [22]:
df_files.head()

,Unnamed: 0,id,filename,company,file_path,sentiment,sentences,title
0,0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,NaN,"['', '', 'is a premier asset manager in Asia. ...",\r
1,1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,NaN,['TP is lifted to \nS$6.00 as we roll over val...,()\r
2,2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,NaN,"['', '. \n\n \n19 March 2010, Baku, Azerbaijan...",(Microsoft Word - SOCARKEPPEL.doc)\r
3,3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,NaN,[],\r
4,4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,NaN,['/o Servoo Apportorai \n(Tiong Seng Contracto...,(Microsoft Word - WSHA2011 Winners_190711.doc)\r


In [23]:
def find_sentiment(important_sents,model):
    try:
        #text = extract_text.apply(path)
        #important_sents = text_summarization.apply(text)
        #print(len(important_sents))
        importance = np.mean(sentiment_analyzer.apply(important_sents,model))
        return np.mean(importance)
    except:
        return -999

In [24]:
## Transform into array else loaded as string
from ast import literal_eval
df_files.sentences = df_files.sentences.apply(literal_eval)

In [36]:
model = keras.models.load_model('model.h5')
tqdm().pandas(desc="Progress:")
df_files.sentiment = df_files.progress_apply(lambda row: np.mean(find_sentiment(row['sentences'],model)),axis=1)


0it [00:00, ?it/s]

Progress::   3%|█▊                                                                    | 10/387 [00:07<07:47,  1.24s/it]
Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\maxpr\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\maxpr\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\maxpr\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

Progress:: 100%|█████████████████████████████████████████████████████████████████████| 387/387 [03:24<00:00,  3.20it/s]


In [25]:
df_files

,Unnamed: 0,id,filename,company,file_path,sentiment,sentences,title
0,0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,NaN,"[, , is a premier asset manager in Asia. It is...",\r
1,1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,NaN,[TP is lifted to \nS$6.00 as we roll over valu...,()\r
2,2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,NaN,"[, . \n\n \n19 March 2010, Baku, Azerbaijan\n ...",(Microsoft Word - SOCARKEPPEL.doc)\r
3,3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,NaN,[],\r
4,4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,NaN,[/o Servoo Apportorai \n(Tiong Seng Contractor...,(Microsoft Word - WSHA2011 Winners_190711.doc)\r
5,5,5,sin63964.doc,Keppel,files/Keppel/sin63964.doc,NaN,[\nCONTROL OF NAVIGATION\n28 Collision Regulat...,\r
6,6,6,KOMTechnology%20StrategyJuly03v3.pdf,Keppel,files/Keppel/KOMTechnology%20StrategyJuly03v3.pdf,NaN,"[, AT THE 2003 SECOND QUARTER RESULTS PRESENT...",(Microsoft Word - KOMTechnology StrategyJuly03...
7,7,7,Keppel%20REIT_AR2016.pdf,Keppel,files/Keppel/Keppel%20REIT_AR2016.pdf,NaN,[Unitholdings\n158 \n Corporate Information\n1...,\r
8,8,8,pre_13sep2010_n2.pdf,Keppel,files/Keppel/pre_13sep2010_n2.pdf,NaN,[Kingcheung. \nWorking experience and \noccup...,(http://info.sgx.com/webcorannc.nsf/vwprint_po...
9,9,9,Dec06.xls,Keppel,files/Keppel/Dec06.xls,NaN,[John) \nLYCA PT747.S6 B4 1965 A history of th...,\r


In [26]:
def getSubmission(df):
    dfCleaned = df[df['sentiment']>0.01]
    top = list(dfCleaned.sort_values('sentiment',ascending=False).head(10).id)
    down = list(dfCleaned.sort_values('sentiment',ascending=True).head(10).id)
    return top,down

In [27]:
def createSubCSV(topId,downId):
    with open("sub.csv",'w',newline='') as csvfile:
        spamwritter = csv.writer(csvfile)
        spamwritter.writerow(['id','rank'])
        for i in range(len(df_files.sentiment)):
            if(i in topId):
                spamwritter.writerow([i,(10-topId.index(i))])
            elif(i in downId):
                spamwritter.writerow([i,(downId.index(i)-10)])
            else:
                spamwritter.writerow([i,0])

In [78]:
top,down = getSubmission(df_files)
createSubCSV(top,down)

In [28]:
import name_entity_analyzer
NEdf = df_files.copy()


In [29]:
NEdf

,Unnamed: 0,id,filename,company,file_path,sentiment,sentences,title
0,0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,NaN,"[, , is a premier asset manager in Asia. It is...",\r
1,1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,NaN,[TP is lifted to \nS$6.00 as we roll over valu...,()\r
2,2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,NaN,"[, . \n\n \n19 March 2010, Baku, Azerbaijan\n ...",(Microsoft Word - SOCARKEPPEL.doc)\r
3,3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,NaN,[],\r
4,4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,NaN,[/o Servoo Apportorai \n(Tiong Seng Contractor...,(Microsoft Word - WSHA2011 Winners_190711.doc)\r
5,5,5,sin63964.doc,Keppel,files/Keppel/sin63964.doc,NaN,[\nCONTROL OF NAVIGATION\n28 Collision Regulat...,\r
6,6,6,KOMTechnology%20StrategyJuly03v3.pdf,Keppel,files/Keppel/KOMTechnology%20StrategyJuly03v3.pdf,NaN,"[, AT THE 2003 SECOND QUARTER RESULTS PRESENT...",(Microsoft Word - KOMTechnology StrategyJuly03...
7,7,7,Keppel%20REIT_AR2016.pdf,Keppel,files/Keppel/Keppel%20REIT_AR2016.pdf,NaN,[Unitholdings\n158 \n Corporate Information\n1...,\r
8,8,8,pre_13sep2010_n2.pdf,Keppel,files/Keppel/pre_13sep2010_n2.pdf,NaN,[Kingcheung. \nWorking experience and \noccup...,(http://info.sgx.com/webcorannc.nsf/vwprint_po...
9,9,9,Dec06.xls,Keppel,files/Keppel/Dec06.xls,NaN,[John) \nLYCA PT747.S6 B4 1965 A history of th...,\r


In [30]:
#tqdm.pandas()
#NERSentences = NEdf.progress_apply(lambda row: name_entity_analyzer.apply(row['sentences'],row['company']) ,axis=1)

In [32]:
for i in range(NEdf.sentences.shape[0]):
    NEdf.sentences[i] = name_entity_analyzer.apply(NEdf.sentences[i],NEdf.company[i])
NEdf

8
27
14
0
19
12
11
601
4
0
238
2
177
252
1
1
250
0
1
8
2
1
457
2
170
280
10
0
1
1
21
3
0
6
16
1
1
21
10
7
480
0
5
205
16
285
9
25
154
1
0
1
0
8
3
0
0
74
0
0
42
332
119
231
0
0
1
0
0
419
0
2
4
456
0
12
1
371
24
101
1
125
9
1
38
456
208
1
12
448
16
101
10
7
266
2
1
19
1
0
0
1
15
36
1
0
2
214
218
0
16
1
0
0
1
203
4
0
261
21
453
10
241
80
0
2
0
2
167
7
0
1
9
0
3
3
1
367
10
3
402
494
265
412
0
9
211
0
5
117
0
27
0
0
15
0
1
28
0
0
0
10
201
0
101
10
1
193
58
2
0
0
72
4
0
1
1
0
178
5
43
0
349
0
290
23
0
1
242
0
22
204
83
4
1
1
6
0
9
1
10
11
10
0
23
0
0
3
0
0
1
24
1
0
97
1
55
54
17
21
3
7
0
2
8
0
0
1
14
0
0
0
201
7
66
0
63
0
1
19
0
1
3
13
0
1
5
0
0
0
0
17
0
0
3
6
2
0
10
32
61
1
10
0
0
1
12
12
8
0
120
1
8
9
0
0
2
31
0
16
35
1
1
2
5
15
3
0
0
12
10
5
4
4
258
1
24
6
0
0
1
36
28
0
0
11
13
8
24
0
25
2
0
5
16
1
1
15
0
0
0
0
13
36
9
1
0
9
228
3
0
2
26
0
9
0
1
12
259
26
0
0
0
0
12
0
0
11
49
11
12
0
1
1
13
0
15
0
12
24
8
6
5
24
0
4
0
10
0
8
9
8
18
0
80
0
5
21
6
110
2
1
4
123
5
6
1


,Unnamed: 0,id,filename,company,file_path,sentiment,sentences,title
0,0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,NaN,[is a premier asset manager in Asia. It is the...,\r
1,1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,NaN,[TP is lifted to \nS$6.00 as we roll over valu...,()\r
2,2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,NaN,"[. \n\n \n19 March 2010, Baku, Azerbaijan\n Œ ...",(Microsoft Word - SOCARKEPPEL.doc)\r
3,3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,NaN,[],\r
4,4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,NaN,[/o Servoo Apportorai \n(Tiong Seng Contractor...,(Microsoft Word - WSHA2011 Winners_190711.doc)\r
5,5,5,sin63964.doc,Keppel,files/Keppel/sin63964.doc,NaN,[\nCONTROL OF NAVIGATION\n28 Collision Regulat...,\r
6,6,6,KOMTechnology%20StrategyJuly03v3.pdf,Keppel,files/Keppel/KOMTechnology%20StrategyJuly03v3.pdf,NaN,[ AT THE 2003 SECOND QUARTER RESULTS PRESENTAT...,(Microsoft Word - KOMTechnology StrategyJuly03...
7,7,7,Keppel%20REIT_AR2016.pdf,Keppel,files/Keppel/Keppel%20REIT_AR2016.pdf,NaN,[Unitholdings\n158 \n Corporate Information\n1...,\r
8,8,8,pre_13sep2010_n2.pdf,Keppel,files/Keppel/pre_13sep2010_n2.pdf,NaN,[Kingcheung. \nWorking experience and \noccup...,(http://info.sgx.com/webcorannc.nsf/vwprint_po...
9,9,9,Dec06.xls,Keppel,files/Keppel/Dec06.xls,NaN,[],\r


In [34]:
model = keras.models.load_model('model.h5')
tqdm().pandas(desc="Progress:")
NEdf.sentiment = NEdf.progress_apply(lambda row: np.mean(find_sentiment(row['sentences'],model)),axis=1)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead




0it [00:00, ?it/s]



Progress::   0%|                                                                               | 0/387 [00:00<?, ?it/s]

Progress::   1%|▎                                                                      | 2/387 [00:00<00:48,  8.00it/s]

Progress::   1%|▌                                                                      | 3/387 [00:00<01:06,  5.82it/s]

Progress::   1%|▋                                                                      | 4/387 [00:00<01:11,  5.38it/s]

Progress::   2%|█                                                                      | 6/387 [00:01<01:06,  5.72it/s]

Progress::   2%|█▎                                                                     | 7/387 [00:01<01:07,  5.60it/s]

Progress::   2%|█▍                                                                     | 8/387 [00:01<01:08,  5.52it/s]

Progress::   2%|█▋                                                                     | 9/387 [00:06<10:10,  1.61s/it]

Progress

Progress::  45%|███████████████████████████████▏                                     | 175/387 [02:00<01:26,  2.45it/s]


Progress::  72%|█████████████████████████████████████████████████▋                   | 279/387 [02:30<00:19,  5.64it/s]


Progress::  97%|███████████████████████████████████████████████████████████████████  | 376/387 [02:51<00:02,  4.23it/s]


Progress:: 100%|█████████████████████████████████████████████████████████████████████| 387/387 [02:54<00:00,  3.01it/s]


In [35]:
NEdf

,Unnamed: 0,id,filename,company,file_path,sentiment,sentences,title
0,0,0,JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf,Keppel,files/Keppel/JD_KDCREIT%20-%20Asset%20Mgt_Anal...,0.576412,[is a premier asset manager in Asia. It is the...,\r
1,1,1,5f4c7c69049c32670176932fd1d305271372e912.pdf,Keppel,files/Keppel/5f4c7c69049c32670176932fd1d305271...,0.683978,[TP is lifted to \nS$6.00 as we roll over valu...,()\r
2,2,2,20100319_214216_BN4_F6859A6425BC8427482576EB00...,Keppel,files/Keppel/20100319_214216_BN4_F6859A6425BC8...,0.487057,"[. \n\n \n19 March 2010, Baku, Azerbaijan\n Œ ...",(Microsoft Word - SOCARKEPPEL.doc)\r
3,3,3,businessassociatelist--1-.xlsx,Keppel,files/Keppel/businessassociatelist--1-.xlsx,-999.000000,[],\r
4,4,4,WSHA2011%20Winners_8%20Aug.pdf,Keppel,files/Keppel/WSHA2011%20Winners_8%20Aug.pdf,0.788647,[/o Servoo Apportorai \n(Tiong Seng Contractor...,(Microsoft Word - WSHA2011 Winners_190711.doc)\r
5,5,5,sin63964.doc,Keppel,files/Keppel/sin63964.doc,0.476852,[\nCONTROL OF NAVIGATION\n28 Collision Regulat...,\r
6,6,6,KOMTechnology%20StrategyJuly03v3.pdf,Keppel,files/Keppel/KOMTechnology%20StrategyJuly03v3.pdf,0.630265,[ AT THE 2003 SECOND QUARTER RESULTS PRESENTAT...,(Microsoft Word - KOMTechnology StrategyJuly03...
7,7,7,Keppel%20REIT_AR2016.pdf,Keppel,files/Keppel/Keppel%20REIT_AR2016.pdf,0.569825,[Unitholdings\n158 \n Corporate Information\n1...,\r
8,8,8,pre_13sep2010_n2.pdf,Keppel,files/Keppel/pre_13sep2010_n2.pdf,0.475624,[Kingcheung. \nWorking experience and \noccup...,(http://info.sgx.com/webcorannc.nsf/vwprint_po...
9,9,9,Dec06.xls,Keppel,files/Keppel/Dec06.xls,-999.000000,[],\r


In [37]:
##Doesn't change anything
topT,downT = getSubmission(NEdf)
createSubCSV(topT,downT)